In [2]:
# Import Dataset from Kaggle
!pip install --upgrade kagglehub
import kagglehub
import pandas as pd
import json
import csv
import os
import sqlite3


# Download latest version
path = kagglehub.dataset_download("s3programmer/disease-diagnosis-dataset")

print("Path to dataset files:", path)
files = os.listdir(path)
print("Files in dataset:", files)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.2
    Uninstalling kagglehub-0.3.2:
      Successfully uninstalled kagglehub-0.3.2


100%|██████████| 32.8k/32.8k [00:00<00:00, 37.4MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/s3programmer/disease-diagnosis-dataset/versions/1
Files in dataset: ['disease_diagnosis.csv']


In [3]:
# Loaded CSV from kaggle
csv_data = os.path.join(path, "disease_diagnosis.csv")
df = pd.read_csv(csv_data)
print(df.head())

   Patient_ID  Age  Gender            Symptom_1    Symptom_2 Symptom_3  \
0           1   74    Male              Fatigue  Sore throat     Fever   
1           2   66  Female          Sore throat      Fatigue     Cough   
2           3   32    Male            Body ache  Sore throat   Fatigue   
3           4   21  Female  Shortness of breath     Headache     Cough   
4           5   53    Male           Runny nose  Sore throat   Fatigue   

   Heart_Rate_bpm  Body_Temperature_C Blood_Pressure_mmHg  \
0              69                39.4              132/91   
1              95                39.0              174/98   
2              77                36.8              136/60   
3              72                38.9              147/82   
4             100                36.6             109/106   

   Oxygen_Saturation_% Diagnosis  Severity       Treatment_Plan  
0                   94       Flu  Moderate  Medication and rest  
1                   98   Healthy      Mild      Rest and

In [4]:
# Modifying CSV file BEFORE converting
# (selected 3 columns and added NEW health risk column)

def mod_columns(df, selected_columns=None, new_col_name=None, new_col_value=None):
    if three_selected_columns:
        df = df[selected_columns].copy()
        # Calculated a health risk score based on age and diagnosis
    if 'Age' in df.columns and 'Diagnosis' in df.columns:
        df['Risk_Score'] = df.apply(lambda row: row['Age'] * (2 if 'Severe' in row['Diagnosis'] else 1), axis=1)
    return df

three_selected_columns = ['Age', 'Diagnosis', 'Gender']


df_modified = mod_columns(df, three_selected_columns)
print("Modified DataFrame:")
print(df_modified.head())

Modified DataFrame:
   Age Diagnosis  Gender  Risk_Score
0   74       Flu    Male          74
1   66   Healthy  Female          66
2   32   Healthy    Male          32
3   21   Healthy  Female          21
4   53   Healthy    Male          53


In [82]:
#function to convert CSV to JSON
def convert_to_json(df, output_file):
    df.to_json(output_file, orient='records', lines=True)
    print(f"DataFrame converted to JSON and saved to {output_file}.")

    #JSON file
    print("\nDisplaying the converted JSON data:")
    with open(output_file, 'r') as f:
        json_data = f.read()
        print(json_data)

# function to save CSV to SQL
def convert_to_sql(df, db_name, table_name):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.commit()

    print(f"DataFrame saved to SQLite database {db_name}, table: {table_name}.")

    #SQL table
    print("\nDisplaying the data from the SQL table:")
    query = f"SELECT * FROM {table_name} LIMIT 5;"
    df_sql = pd.read_sql(query, conn)
    print(df_sql.head())

    conn.close()

#processor to choose the output format
def processor_data(input_file, output_format, output_file=None, db_name=None, table_name=None):
    df = pd.read_csv(input_file)
    if output_format == 'json':
        if output_file is None:
            output_file = 'output_data.json'
        convert_to_json(df_modified, output_file)

    elif output_format == 'sql':
        if db_name is None:
            db_name = 'output_database.db'
        if table_name is None:
            table_name = 'output_table'
        convert_to_sql(df_modified, db_name, table_name)

    else:
        print("Invalid output format. Choose either 'json' or 'sql'.")

# JSON OR SQL option
output_format_json = 'json'
output_format_sql = 'sql'

#convert to JSON and display:
processor_data(csv_file, output_format_json, output_file=output_json_file)


DataFrame converted to JSON and saved to disease_file.json.

Displaying the converted JSON data:
{"Age":74,"Diagnosis":"Flu","Gender":"Male","Risk_Score":74}
{"Age":66,"Diagnosis":"Healthy","Gender":"Female","Risk_Score":66}
{"Age":32,"Diagnosis":"Healthy","Gender":"Male","Risk_Score":32}
{"Age":21,"Diagnosis":"Healthy","Gender":"Female","Risk_Score":21}
{"Age":53,"Diagnosis":"Healthy","Gender":"Male","Risk_Score":53}
{"Age":22,"Diagnosis":"Flu","Gender":"Male","Risk_Score":22}
{"Age":21,"Diagnosis":"Bronchitis","Gender":"Male","Risk_Score":21}
{"Age":71,"Diagnosis":"Healthy","Gender":"Male","Risk_Score":71}
{"Age":56,"Diagnosis":"Cold","Gender":"Female","Risk_Score":56}
{"Age":53,"Diagnosis":"Flu","Gender":"Male","Risk_Score":53}
{"Age":72,"Diagnosis":"Healthy","Gender":"Female","Risk_Score":72}
{"Age":28,"Diagnosis":"Cold","Gender":"Female","Risk_Score":28}
{"Age":78,"Diagnosis":"Healthy","Gender":"Male","Risk_Score":78}
{"Age":23,"Diagnosis":"Bronchitis","Gender":"Male","Risk_Score"

In [34]:
#converted to JSON display only first rows since above file is huge^
transit_json = df.to_json(orient='records', indent=4)
transit_deserialized = json.loads(transit_json)
print(json.dumps(transit_deserialized[:5], indent=4))

[
    {
        "Patient_ID": 1,
        "Age": 74,
        "Gender": "Male",
        "Symptom_1": "Fatigue",
        "Symptom_2": "Sore throat",
        "Symptom_3": "Fever",
        "Heart_Rate_bpm": 69,
        "Body_Temperature_C": 39.4,
        "Blood_Pressure_mmHg": "132/91",
        "Oxygen_Saturation_%": 94,
        "Diagnosis": "Flu",
        "Severity": "Moderate",
        "Treatment_Plan": "Medication and rest"
    },
    {
        "Patient_ID": 2,
        "Age": 66,
        "Gender": "Female",
        "Symptom_1": "Sore throat",
        "Symptom_2": "Fatigue",
        "Symptom_3": "Cough",
        "Heart_Rate_bpm": 95,
        "Body_Temperature_C": 39.0,
        "Blood_Pressure_mmHg": "174/98",
        "Oxygen_Saturation_%": 98,
        "Diagnosis": "Healthy",
        "Severity": "Mild",
        "Treatment_Plan": "Rest and fluids"
    },
    {
        "Patient_ID": 3,
        "Age": 32,
        "Gender": "Male",
        "Symptom_1": "Body ache",
        "Symptom_2": "Sore t

In [97]:
#convert CSV to SQL display:
process_data(csv_file, output_format_sql, db_name=db_name, table_name=table_name)

DataFrame saved to SQLite database disease_database.db, table: disease_table.

Displaying the data from the SQL table:
   Age Diagnosis  Gender  Risk_Score
0   74       Flu    Male          74
1   66   Healthy  Female          66
2   32   Healthy    Male          32
3   21   Healthy  Female          21
4   53   Healthy    Male          53


In [41]:
# JSON Dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rmisra/news-category-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/rmisra/news-category-dataset/versions/3


In [59]:
#JSON to dataframe

json_data = os.path.join(path, "News_Category_Dataset_v3.json")
df_json = pd.read_json(json_data, lines=True)

print(df_json.head())


                                                link  \
0  https://www.huffpost.com/entry/covid-boosters-...   
1  https://www.huffpost.com/entry/american-airlin...   
2  https://www.huffpost.com/entry/funniest-tweets...   
3  https://www.huffpost.com/entry/funniest-parent...   
4  https://www.huffpost.com/entry/amy-cooper-lose...   

                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog y

In [60]:
# modifying JSON file (dropping and adding new column)
df_json.drop(columns=['link'], inplace=True)

print("Dropped column: 'link'")

# new column 'Source_Type'
df_json['Source_Type'] = 'News'

# modified DataFrame
print("Modified DataFrame:")
print(df_json.head())

Dropped column: 'link'
Modified DataFrame:
                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog you don't understand wha...         Elyse Wanshel   
3  "Accidentally put grown-up toothpaste on my to...      Caroline Bologna   
4  Amy Cooper accused investment firm Franklin Te...        Nina Golgowski   

        date Source_Type  
0 2022-09-23        News  
1 2022-09-23        News  
2 20

In [61]:
#JSON to CSV
csv_output_file = 'news_category.csv'
df_json.to_csv(csv_output_file, index=False)

print(f"JSON file converted to CSV and saved to {csv_output_file}.")
print(df_json.head())

JSON file converted to CSV and saved to news_category.csv.
                                            headline   category  \
0  Over 4 Million Americans Roll Up Sleeves For O...  U.S. NEWS   
1  American Airlines Flyer Charged, Banned For Li...  U.S. NEWS   
2  23 Of The Funniest Tweets About Cats And Dogs ...     COMEDY   
3  The Funniest Tweets From Parents This Week (Se...  PARENTING   
4  Woman Who Called Cops On Black Bird-Watcher Lo...  U.S. NEWS   

                                   short_description               authors  \
0  Health experts said it is too early to predict...  Carla K. Johnson, AP   
1  He was subdued by passengers and crew when he ...        Mary Papenfuss   
2  "Until you have a dog you don't understand wha...         Elyse Wanshel   
3  "Accidentally put grown-up toothpaste on my to...      Caroline Bologna   
4  Amy Cooper accused investment firm Franklin Te...        Nina Golgowski   

        date Source_Type  
0 2022-09-23        News  
1 2022-09-23   

In [70]:
#into SQL database file
db_file = 'news_category.db'
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS news_category (
    category TEXT,
    headline TEXT,
    authors TEXT,
    source_type TEXT -- Adjust based on actual CSV columns
)
''')

#CSV data into the table
import csv

with open('news_category.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        cursor.execute('''
        INSERT INTO news_category (category, headline, authors, source_type)
        VALUES (?, ?, ?, ?)
        ''', (row['category'], row['headline'], row['authors'], row['Source_Type']))

rows = cursor.fetchall()

#commit and close connection
db_file = 'news_category.db'
conn = sqlite3.connect(db_file)
cursor = conn.cursor()
cursor.execute('SELECT * FROM news_category LIMIT 10')
rows = cursor.fetchall()

#display
print("Data from 'news_category' table:")
for row in rows:
    print(row)

#close the connection
conn.close()

Data from 'news_category' table:
('U.S. NEWS', 'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters', 'Carla K. Johnson, AP', 'News')
('U.S. NEWS', 'American Airlines Flyer Charged, Banned For Life After Punching Flight Attendant On Video', 'Mary Papenfuss', 'News')
('COMEDY', '23 Of The Funniest Tweets About Cats And Dogs This Week (Sept. 17-23)', 'Elyse Wanshel', 'News')
('PARENTING', 'The Funniest Tweets From Parents This Week (Sept. 17-23)', 'Caroline Bologna', 'News')
('U.S. NEWS', 'Woman Who Called Cops On Black Bird-Watcher Loses Lawsuit Against Ex-Employer', 'Nina Golgowski', 'News')
('U.S. NEWS', 'Cleaner Was Dead In Belk Bathroom For 4 Days Before Body Found: Police', '', 'News')
('U.S. NEWS', 'Reporter Gets Adorable Surprise From Her Boyfriend While Live On TV', 'Elyse Wanshel', 'News')
('WORLD NEWS', 'Puerto Ricans Desperate For Water After Hurricane Fiona’s Rampage', 'DÁNICA COTO, AP', 'News')
('CULTURE & ARTS', 'How A New Documentary Captures The 

In [79]:
#data ingestion summary for SECOND dataset (JSON to CSV into SQL)
csv_output_file = 'news_category.csv'

df_json = pd.read_csv(csv_output_file)

#summary function
def generate_summary(df, stage):
    num_records = df.shape[0]
    num_columns = df.shape[1]
    print(f"{stage} Summary:")
    print(f"Number of records: {num_records}")
    print(f"Number of columns: {num_columns}")
    print("="*30)

#summary after ingestion
generate_summary(df_json, "Ingestion")

#summary after modification
generate_summary(df_json, "Post-Processing")

#modified DataFrame to SQL Database
db_file = 'news_category.db'
conn = sqlite3.connect(db_file)
df_json.to_sql('news_category', conn, if_exists='replace', index=False)

print(f"Data saved to SQL database at {db_file}.")
conn.close()

Ingestion Summary:
Number of records: 209527
Number of columns: 6
Post-Processing Summary:
Number of records: 209527
Number of columns: 6
Data saved to SQL database at news_category.db.


In [94]:
#data ingestion summary for FIRST dataset (CSV to JSON into SQL)
csv_file_path = os.path.join(path, 'disease_diagnosis.csv')
df_disease = pd.read_csv(csv_file_path)

def generate_summary(df, stage):
    num_records = df.shape[0]
    num_columns = df.shape[1]
    print(f"{stage} Summary:")
    print(f"Number of records: {num_records}")
    print(f"Number of columns: {num_columns}")
    print("=" * 30)

#summary after ingestion for the disease dataset
generate_summary(df_disease, "Ingestion")

three_selected_columns = ['Age', 'Diagnosis', 'Gender']
df_modified = mod_columns(df_disease, three_selected_columns)

#summary after modifications
generate_summary(df_modified, "Post-Processing")

Ingestion Summary:
Number of records: 2000
Number of columns: 13
Post-Processing Summary:
Number of records: 2000
Number of columns: 4
